# Zero-Shot Learning

### Installing the modules

In [ ]:
!pip install numpy shap transformers lime

### Loading the modules

In [4]:
import pandas as pd
import numpy as np
import shap
import transformers
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer, ZeroShotClassificationPipeline

### Dataset

### Models

In [ ]:
#classifier_bart = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
#classifier_distil = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3")
#classifier_bert = pipeline("zero-shot-classification", model="typeform/distilbert-base-uncased-mnli")

In [ ]:
# Loading the hugging face model and tokenizer

In [ ]:
model_bart = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")
tokenizer_bart = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

In [ ]:
model_distil = AutoModelForSequenceClassification.from_pretrained("valhalla/distilbart-mnli-12-3")
tokenizer_distil = AutoTokenizer.from_pretrained("valhalla/distilbart-mnli-12-3")

In [ ]:
model_bert = AutoModelForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli")
tokenizer_bert = AutoTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")

In [ ]:
# Create a custom pipeline that only requires the text parameter 
# for the __call__ method and provides a method to set the labels
class ZeroShotModelPipeline(ZeroShotClassificationPipeline):
    # Overwrite the __call__ method
    def __call__(self, *args):
        out = super().__call__(args[0], self.set_labels)[0]

        return [[{"label":x[0], "score": x[1]}  for x in zip(out["labels"], out["scores"])]]

    def set_labels(self, labels: Union[str,List[str]]):
        self.set_labels = labels

In [ ]:
def score_and_visualize(text, shap_values):
    prediction = pipe(text)
    print(f"Model predictions are: {prediction}")
    shap.plots.text(shap_values)

In [ ]:
text = ["I love playing cricket!"]
labels = ["insect","sports", "animal"]

In [ ]:
# Assign the labels for the classification model
model_bart.config.label2id.update({v:k for k,v in enumerate(labels)})
model_bart.config.id2label.update({k:v for k,v in enumerate(labels)})

model_distil.config.label2id.update({v:k for k,v in enumerate(labels)})
model_distil.config.id2label.update({k:v for k,v in enumerate(labels)})

model_bert.config.label2id.update({v:k for k,v in enumerate(labels)})
model_bert.config.id2label.update({k:v for k,v in enumerate(labels)})

In [ ]:
pipe1 = ZeroShotModelPipeline(model=model_bart, tokenizer=tokenizer_bart, return_all_scores=True)
pipe1.set_labels(labels)

pipe2 = ZeroShotModelPipeline(model=model_distil, tokenizer=tokenizer_distil, return_all_scores=True)
pipe2.set_labels(labels)

pipe3 = ZeroShotModelPipeline(model=model_bert, tokenizer=tokenizer_bert, return_all_scores=True)
pipe3.set_labels(labels)

## Model Explainability

In [ ]:
# SHAP Explainer
explainer1 = shap.Explainer(pipe1)
shap_values1 = explainer1(text)

explainer2 = shap.Explainer(pipe2)
shap_values2 = explainer2(text)

explainer3 = shap.Explainer(pipe3)
shap_values3 = explainer3(text)

In [ ]:
score_and_visualize(text, shap_values1)

In [ ]:
score_and_visualize(text, shap_values2)

In [ ]:
score_and_visualize(text, shap_values3)